# Data preprocessor

In [25]:
import os
import glob
import cv2
import pandas as pd

from os import mkdir

def load_img_paths(target):
    '''
    Retrieve the full path of all images in the dataset
    '''
    return glob.glob(target + '/*.tif')

output_dir = '../data/learning'
try: 
    mkdir(output_dir)
except FileExistsError:
    print('directory already exists')

data_dir = r'../data'
original_data_dir = data_dir + ('/learning')
all_files = load_img_paths(original_data_dir)
all_files[:5]

directory already exists


['../data/learning\\Tp26_Y000_X000_040.tif',
 '../data/learning\\Tp26_Y000_X001_040.tif',
 '../data/learning\\Tp26_Y000_X002_040.tif',
 '../data/learning\\Tp26_Y000_X003_040.tif',
 '../data/learning\\Tp26_Y000_X004_040.tif']

In [20]:
# For learning folder
def image_pair_coords(fname, ):
    '''
    get an image's pair in one for four directions
    '''
    im_coords = path.split(fname)[-1] \
                        .split('.')[0]    \
                        .split('_')[1:3]
    good_direction = False
    while not good_direction:
        #     direction = np.random.randint(0,4)
        direction = np.random.choice([1,3]) # horizontal only
        y_part = int(im_coords[0].split('Y')[1])
        x_part = int(im_coords[1].split('X')[1])
        if direction == 0:
            y_part -=1
        elif direction == 1:
            x_part +=1
        elif direction == 2:
            y_part +=1
        else: #direction == 3:
            x_part -=1
            
        if x_part >= xmin and x_part <= xmax and \
           y_part >= ymin and y_part <= ymax:
            good_direction = True

    ystr = str(y_part).rjust(2,'0')
    xstr = str(x_part).rjust(2,'0')
    fpath = path.split(fname)[:-1][0]
    next_fname = path.join(fpath, 'Tp26_Y0%s_X0%s_040.tif' % (ystr, xstr))
    return next_fname       

# For learning2 folder, or anything generated from the im_split script
def image_pair_coords_learning2(fname):
    im_id = path.split(fname)[-1]         \
                        .split('.')[0]    \
                        .split('_')[-1]
    image_max = max(map(lambda x: int(x.split('.')[-2].split('_')[-1]), train_paths))
    rows = int(math.sqrt(image_max))
    im_id = int(im_id)
    target_id = -1
    good_direction = False
    while not good_direction:
        direction = np.random.randint(0,4)
        if direction == 0: # right
            target_id = im_id-1
        elif direction == 1: # top
            target_id = im_id-rows
        elif direction == 2: # left 
            target_id = im_id+1
        else: #direction == 3: # bottom
            target_id = im_id+rows
        
        # left edge image cannot have right direction, right edge image cannot have left dir.
        if (im_id % rows == 0 and direction == 2) or \
           (im_id % rows == 1 and direction == 0): 
            good_direction = False
        elif target_id > 0 and target_id <= image_max:
            good_direction = True
            
    fpath = path.split(fname)[:-1][0]
    next_fname = path.join(fpath, 'T1_segment_%d.tif' % (target_id))
    return next_fname

In [27]:
import os

df = pd.DataFrame(columns=['ref', 'mov'])

ymin=0
ymax=7

xmin=0
xmax=14

for f in all_files:
    fname = os.path.basename(f)
    fpath = os.path.split(fname)[:-1][0]

    y,x = os.path.split(fname)[-1]  \
                        .split('.')[0] \
                        .split('_')[1:3]
    x = int(x[1:])
    y = int(y[1:])
    
    if x < xmax and x > xmin:
        coords = [x+1, x-1]
    elif x == xmax:
        coords = [x-1]
    else: # x == xmin
        coords = [x+1]
    for x in coords:
        ystr = str(y).rjust(2,'0')
        xstr = str(x).rjust(2,'0')
        
        next_fname = os.path.join(fpath, 'Tp26_Y0%s_X0%s_040.tif' % (ystr, xstr))
        df = df.append({'ref':fname, 'mov':next_fname}, ignore_index=True)

df.to_csv(os.path.join(output_dir, 'horiz_pairs.csv'))

In [28]:
df.head()

,ref,mov
0,Tp26_Y000_X000_040.tif,Tp26_Y000_X001_040.tif
1,Tp26_Y000_X001_040.tif,Tp26_Y000_X002_040.tif
2,Tp26_Y000_X001_040.tif,Tp26_Y000_X000_040.tif
3,Tp26_Y000_X002_040.tif,Tp26_Y000_X003_040.tif
4,Tp26_Y000_X002_040.tif,Tp26_Y000_X001_040.tif


In [29]:
len(df)

224